In [1]:
import pickle

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [5]:
import mlflow

In [2]:
from sklearn.pipeline import make_pipeline

In [10]:

mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [9]:
mlflow.search_experiments()

[<Experiment: artifact_location='file:///C:/Users/aksha/artifacts/0', creation_time=1720629927114, experiment_id='0', last_update_time=1720629927114, lifecycle_stage='active', name='Default', tags={}>]

In [11]:
mlflow.set_experiment("green-taxi-duration-random-forest")

2024/07/10 20:48:58 INFO mlflow.tracking.fluent: Experiment with name 'green-taxi-duration-random-forest' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///C:/Users/aksha/artifacts/1', creation_time=1720630138167, experiment_id='1', last_update_time=1720630138167, lifecycle_stage='active', name='green-taxi-duration-random-forest', tags={}>

In [15]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [17]:
df_train = read_dataframe('C:/Users/aksha/Documents/Study/mlops-zero-to-one/02-Experiment Tracking/data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('C:/Users/aksha/Documents/Study/mlops-zero-to-one/02-Experiment Tracking/data/green_tripdata_2021-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [18]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="model")

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 6.7558229919200725


C:\Users\aksha\anaconda3\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [19]:
from mlflow.tracking import MlflowClient


In [20]:
MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000'
RUN_ID = '410d302e047e4a389cd0ad29458f327c'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [22]:
with open(path, 'rb') as f_out:
    dv = pickle.load(f_out)

In [23]:
dv

DictVectorizer()